# 自动微分

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#一个简单的例子" data-toc-modified-id="一个简单的例子-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>一个简单的例子</a></span></li><li><span><a href="#非标量变量的反向传播" data-toc-modified-id="非标量变量的反向传播-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>非标量变量的反向传播</a></span></li><li><span><a href="#分离计算" data-toc-modified-id="分离计算-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>分离计算</a></span></li><li><span><a href="#Python控制流的梯度计算" data-toc-modified-id="Python控制流的梯度计算-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Python控制流的梯度计算</a></span></li></ul></div>

求导是⼏乎所有深度学习优化算法的关键步骤。虽然求导的计算很简单，只需要⼀些基本的微积分。但对于复杂的模型，⼿⼯进⾏更新是⼀件很痛苦的事情（⽽且经常容易出错）。

深度学习框架通过⾃动计算导数，即⾃动微分（automatic differentiation）来加快求导。实际中，根据设计好的模型，系统会构建⼀个计算图（computational graph），来跟踪计算是哪些数据通过哪些操作组合起来产⽣输出。⾃动微分使系统能够随后反向传播梯度。这⾥，反向传播（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

### 一个简单的例子

作为⼀个演⽰例⼦，假设我们想对函数$y=2\textbf{x}^\top\textbf{x}$关于列向量$\textbf{x}$求导。⾸先，我们创建变量x并为其分配⼀个初始值。

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算y关于$\textbf{x}$的梯度之前，需要⼀个地⽅来存储梯度。重要的是，我们不会在每次对⼀个参数求导时都分配新的内存。因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。注意，**⼀个标量函数关于向量$\textbf{x}$的梯度是向量，并且与$\textbf{x}$具有相同的形状**。

In [2]:
x.requires_grad_(True) # 等价x = torch.arange(4.0, requires_grad=True)
x.grad # 默认值是None

现在计算$y$。

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

x是⼀个⻓度为4的向量，计算x和x的点积，得到了我们赋值给y的标量输出。接下来，通过调⽤反向传播函数来⾃动计算y关于x每个分量的梯度，并打印这些梯度。

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函数$y=2\textbf{x}^\top\textbf{x}$关于$\textbf{x}$的梯度应为$4\textbf{x}$。让我们快速验证这个梯度是否计算正确。

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算x的另⼀个函数。

In [6]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

### 非标量变量的反向传播

当y不是标量时，向量y关于向量x的导数的最⾃然解释是⼀个矩阵。对于⾼阶和⾼维的y和x，求导的结果可以是⼀个⾼阶张量。

然⽽，虽然这些更奇特的对象确实出现在⾼级机器学习中（包括深度学习中），但当调⽤向量的反向计算时，我们通常会试图计算⼀批训练样本中每个组成部分的损失函数的导数。这⾥，我们的⽬的不是计算微分矩阵，⽽是单独计算批量中每个样本的偏导数之和。

In [7]:
# 对⾮标量调⽤backward需要传⼊⼀个gradient参数，该参数指定微分函数关于self的梯度。 
# 本例只想求偏导数的和，所以传递⼀个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

### 分离计算

有时，我们希望将某些计算移动到记录的计算图之外。例如，假设y是作为x的函数计算的，⽽z则是作为y和x的函数计算的。想象⼀下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为⼀个常数，并且只考虑到x在y被计算后发挥的作⽤。

这⾥可以分离y来返回⼀个新变量u，该变量与y具有相同的值，但丢弃计算图中如何计算y的任何信息。换句话说，梯度不会向后流经u到x。因此，下⾯的反向传播函数计算$z=u*x$关于x的偏导数，同时将u作为常数处理，⽽不是$z=x*x*x$关于x的偏导数。

In [8]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

由于记录了y的计算结果，我们可以随后在y上调⽤反向传播，得到$y=x*x$关于的x的导数，即$2*x$。

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

### Python控制流的梯度计算

使⽤⾃动微分的⼀个好处是：即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调⽤），我们仍然可以计算得到的变量的梯度。在下⾯的代码中，while循环的迭代次数和if语句的结果都取决于输⼊a的值。

In [10]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

计算梯度：

In [11]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

我们现在可以分析上⾯定义的$f$函数。请注意，它在其输⼊a中是分段线性的。换⾔之，对于任何a，存在某个常量标量k，使得$f(a)=k*a$，其中k的值取决于输⼊a，因此可以⽤d/a验证梯度是否正确。

In [12]:
a.grad == d / a

tensor(True)